# Context Manager
---
**Syntax:**
```py
with ctx() as cx:
    pass
    # code to execute
```

**Exaple:**
```py
with open('data.csv') as f:
    for line in f.lines():
        print(line)
## f: closes automatically ##
```

**_Loose translation:_**
```py
cx = ctx()
cx.__enter__()
try:
    pass
    # code to execute
finally:
    cx.__exit__()
```

**_To create a context manager:_**
```py
class ctx:
    def __enter__(self):
        pass # initial operations like connect to internet
    
    def __exit__(self, *_, **__):
        pass # teardown operations like close the connection
             # this executes if __enter__ is success...
```

#### Example:

In [18]:
from sqlite3 import connect

In [19]:
class temptable:
    def __init__(self, cursor):
        self.cursor = cursor
        
    def __enter__(self):
        self.cursor.execute('create table temp_table (x int, y int);')
        for x in range(10):
            self.cursor.execute('insert into temp_table (x, y) values ({x}, {y});'.format(x=x, y=x * x))
    
    def __exit__(self, *_, **__):
        self.cursor.execute('drop table temp_table;')

In [20]:
with connect('datasets/xy.db') as conn:
    cursor = conn.cursor()
    with temptable(cursor): # using the ctx manager
        for row in cursor.execute('select * from temp_table;'):
            print(row)
    cursor.execute('select * from temp_table;') 
    # out of context temptable temp_table already dropped by __exit__

(0, 0)
(1, 1)
(2, 4)
(3, 9)
(4, 16)
(5, 25)
(6, 36)
(7, 49)
(8, 64)
(9, 81)


OperationalError: no such table: temp_table

🔝🔝 `temp_table` not found because 
```py
 cursor.execute('select * from temp_table;')
```
statement is out of `temptable` context `__exit__` function already executed and droped the table


---

## Enforcing same functionality with generators to ensure sequencing
#### Example:

In [21]:
from sqlite3 import connect

In [50]:
def fresh_table(cursor):
    cursor.execute('create table fresh_table (x int, y int);')
    for x in range(10):
        cursor.execute('insert into fresh_table (x, y) values ({x}, {y});'.format(x=x, y=x*(x+2)))
    #------ created table fresh_table and insert some data -----#
    
    yield
    
    cursor.execute('drop table fresh_table;')
    #------ drop table fresh_table -----#

In [51]:
class fresh_table_ctx:
    def __init__(self, generator_fun):
        self.generator_fun = generator_fun
    
    def __call__(self, cursor):
        self.cursor = cursor
        return self
    
    def __enter__(self):
        self.g = self.generator_fun(self.cursor)
        next(self.g)
    
    def __exit__(self, *_, **__):
        try:
            next(self.g)
        except StopIteration:
            pass

In [52]:
fresh_table = fresh_table_ctx(fresh_table)

🔝🔝**This is a `decorator` syntax which is equal to**
```py
@fresh_table_ctx
def fresh_table(cursor):
    cursor.execute('create table fresh_table (x int, y int);')
    for x in range(10):
        cursor.execute('insert into fresh_table (x, y) values ({x}, {y});'.format(x=x, y=x*(x+2)))
    yield
    cursor.execute('drop table fresh_table;')
```

In [55]:
with connect('datasets/xy.db') as db:
    cursor = db.cursor()
    with fresh_table(cursor):
        rows = cursor.execute('select * from fresh_table;')
        for row in rows:
            print(row)
    cursor.execute('select * from fresh_table;') # out of context

(0, 0)
(1, 3)
(2, 8)
(3, 15)
(4, 24)
(5, 35)
(6, 48)
(7, 63)
(8, 80)
(9, 99)


OperationalError: no such table: fresh_table

**Summary:**
> A generator ensures the sequance of operations

> `__enter__` and `__exit__` creates the context manager

> `generator = ctx_manager(generator)` combines the required functionality

**For all of this `python` has in-built decorator**
#### Example

In [56]:
from sqlite3 import connect
from contextlib import contextmanager

In [58]:
@contextmanager #--------> no need to write class fresh_table_ctx
def final_table(cursor):
    cursor.execute('create table final_table (x int, y int);')
    for x in range(10):
        cursor.execute('insert into final_table (x, y) values ({x}, {y});'.format(x=x, y=x*(x+2)))

    yield
    cursor.execute('drop table final_table;')

In [60]:
with connect('datasets/xy.db') as db:
    cursor = db.cursor()
    with final_table(cursor):
        rows = cursor.execute('select * from final_table;')
        for row in rows:
            print(row)
    cursor.execute('select * from final_table;') # out of context

(0, 0)
(1, 3)
(2, 8)
(3, 15)
(4, 24)
(5, 35)
(6, 48)
(7, 63)
(8, 80)
(9, 99)


OperationalError: no such table: final_table